# Diabetes database binary classification

### Initialization

Importing modules

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
import pandas as pd
import tensorflow as tf
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Some architectural and training hyperparameters

In [2]:
activation = "gelu"
predecessor_level_connection_affinity_factor_first = 19.613
predecessor_level_connection_affinity_factor_main = 0.5518
max_consecutive_lateral_connections = 34
p_lateral_connection = 0.36014
num_lateral_connection_tries_per_unit = 11
learning_rate = 0.095
epochs = 145
batch_size = 100
maximum_levels = 5
maximum_units_per_level = 5
maximum_neurons_per_unit = 25

### Importing raw data, preparing the training set

In [3]:
# Loading raw data
raw_data = pd.read_csv('diabetes.csv')

# Splitting off the labels
label = raw_data.pop('Outcome').astype(int).values

# Converting the rest into numerical values
data_numeric = raw_data.astype(float)
data_np = data_numeric.values

# Logging 
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_diabetes_test'

# Labelled train datase
tensor_x = tf.constant(data_np)
training_x = [tensor_x]
train_labels = [label]

# Define input shapes of the model
INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]

# Define output shapes (binary 0/1 class)
OUTPUT_SHAPES = [1]  

### Running Cerebros AutoML session

Instantiate Cerebros search

In [4]:
meta_trial_number = 42 # can be anything unless in distributed training
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.35,
    direction='maximize',
    metric_to_rank_by='val_binary_accuracy',
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='sigmoid',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=5,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(),
             tf.keras.metrics.Precision(),
             tf.keras.metrics.Recall()],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number)
#

Best model search

In [5]:
result = cerebros_automl.run_random_search()
best_model_found = cerebros_automl.get_best_model()

SimpleCerebrosRandomSearch.input_shapes: [8]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'25': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'21': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'5': <class 'cerebros.units.units.DenseUnit'>}]
3
Trying to create level 3
We think level 3's predecessors are: [0, 1, 2]
k is: 4 value is: [{'3': <class 'cerebros.units.units.DenseUnit'>}, {'15': <class 'cerebros.units.units.DenseUnit'>}, {'6': <class 'cerebros.units.units.DenseUnit'>}]
4
Trying to create level 4
We think level 4's predecessors are: [0, 1, 2, 3]
k is: 5 value is: [{'1': <class 'cerebros.units.units.FinalDenseUnit'>}]
5
Trying to create Final level 5
Trying to create level 5
We think level final level 5

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000005_tr_0_FinalDenseUnit_0000000000000005_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_0_dns_')>, <KerasTensor: shape=(None, 15) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_1_dns_')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_2_dns_')>, <KerasTensor: shape=(None, 15) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000004_tr_0_DenseUnit_0000000000000004_tr_0_1_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer '

INFO:tensorflow:Assets written to: 2023_10_18_05_08_cerebros_auto_ml_test_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
         loss  binary_accuracy  precision    recall      val_loss  \
0    1.329748         0.529058   0.375587  0.439560  11822.198242   
1    0.734515         0.693387   0.592357  0.510989   3091.223633   
2    0.583337         0.723447   0.619565  0.626374   1292.716064   
3    0.603378         0.685371   0.588652  0.456044    330.483856   
4    0.549748         0.723447   0.619565  0.626374    310.223267   
..        ...              ...        ...       ...           ...   
140  0.166708         0.925852   0.900553  0.895604      1.452585   
141  0.209200         0.909820   0.900585  0.846154      1.489020   
142  0.207995         0.889780   0.880240  0.807692      1.685990   
143  0.207271         0.921844   0.899441  0.884615      1.455172   
144  0.211071         0.911824   0.855670  0.912088      1.641788   

     val_binary_accuracy  val_precision  val_recall  trial_number  \
0               0.319703       0.319703    1.000000             0   
1      

inputs
KerasTensor(type_spec=TensorSpec(shape=(None, 8), dtype=tf.float32, name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'), name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'")

outputs
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000005_tr_0_FinalDenseUnit_0000000000000005_tr_0_0_dns_/Sigmoid:0', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000005_tr_0_FinalDenseUnit_0000000000000005_tr_0_0_dns_'")
Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
_______________________________________________________________________________

...

INFO:tensorflow:Assets written to: 2023_10_18_05_08_cerebros_auto_ml_test_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
         loss  binary_accuracy  precision    recall    val_loss  \
0    1.177161         0.639323   0.485149  0.548507  568.941162   
1    0.781704         0.695391   0.581522  0.587912  100.404579   
2    0.645214         0.687375   0.584416  0.494505    7.713130   
3    0.572339         0.753507   0.648241  0.708791   24.413729   
4    0.547408         0.747495   0.722222  0.500000   17.845760   
..        ...              ...        ...       ...         ...   
140  0.278524         0.875751   0.916667  0.725275    1.056108   
141  0.315196         0.849699   0.763547  0.851648    1.014859   
142  0.429392         0.835671   0.847222  0.670330    1.060241   
143  0.316002         0.851703   0.778351  0.829670    0.964590   
144  0.320389         0.855711   0.816092  0.780220    0.891377   

     val_binary_accuracy  val_precision  val_recall  trial_number  \
0               0.319703       0.319703    1.000000             0   
1               0.680297       

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000003_tr_0_FinalDenseUnit_0000000000000003_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 13) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_1_dns_')>, <KerasTensor: shape=(None, 13) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_1_dns_')>, <KerasTensor: shape=(None, 13) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_0_dns_')>, <KerasTensor: shape=(None, 13) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_1_dns_')>, <KerasTensor: shape=(None, 8) dtype=float32 (created by layer

INFO:tensorflow:Assets written to: 2023_10_18_05_08_cerebros_auto_ml_test_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


returning trial 0 oracles
         loss  binary_accuracy  precision    recall    val_loss  \
0    1.286850         0.658854   0.510204  0.559702  482.383209   
1    0.895616         0.657315   0.532934  0.489011   19.709755   
2    0.667627         0.745491   0.668712  0.598901   64.158340   
3    0.544948         0.747495   0.664706  0.620879   20.453646   
4    0.536215         0.721443   0.652482  0.505495    3.757691   
..        ...              ...        ...       ...         ...   
140  0.258023         0.891784   0.823232  0.895604    1.289428   
141  0.287437         0.893788   0.900621  0.796703    1.243170   
142  0.333871         0.855711   0.777778  0.846154    1.416956   
143  0.287573         0.877756   0.906040  0.741758    1.253397   
144  0.302018         0.873747   0.787440  0.895604    1.092810   

     val_binary_accuracy  val_precision  val_recall  trial_number  \
0               0.319703       0.319703    1.000000             0   
1               0.382900       

Output the number of parameters, accuracy, etc.

In [6]:
trainable_params = np.sum([np.prod(w.get_shape()) for w in best_model_found.trainable_weights])
non_trainable_params = np.sum([np.prod(w.get_shape()) for w in best_model_found.non_trainable_weights])
total_params = trainable_params + non_trainable_params
#
print(f"Best model found: {total_params} total parameters ({trainable_params} trainable, {non_trainable_params} non-trainable)")
#
print(f"Best accuracy is ({cerebros_automl.metric_to_rank_by}): {result}")

Best model found: 20691 total parameters (17405 trainable, 3286 non-trainable)
Best accuracy is (val_binary_accuracy): 0.8141263723373413
